### one time installations

In [1]:
# !gdown --id 1pb7gEkctrVrJA79EAIo7H7nuzD6uV1fW
# !gdown --id 1oIeAE9HXXKWPcYa-AZ0ht5ef6sKe_Vh_
# !gdown --id 10rAuIDvsYR2yDiCqP7GmYGPc-UmtLbJb

In [2]:
# !pip install --quiet transformers
# !pip install --quiet datasets 
# !pip install --quiet SentencePiece
# !pip install --quiet pytorch-lightning

### libraries

In [1]:
import numpy as np
import pandas as pd
import sklearn
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer

### hyper parameters

In [2]:
class hyperparameters:
    # tokenizer
    tokenizer_name = "deepset/xlm-roberta-large-squad2" # model_name # CHANGE THIS; TRY XLM-ROBERTA
    max_len = 384 # maximum length of context and question in a datapoint
    overlap_len = 128 # overlap between two parts of the context when it is split
    
    # model
    model_name = "deepset/xlm-roberta-large-squad2"
    

In [3]:
hyperparams = hyperparameters()

### data - understanding the pipeline

In [4]:
tokenizer = AutoTokenizer.from_pretrained(hyperparams.tokenizer_name)

In [5]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
# test_df = pd.read_csv('test.csv')
# sample_df = pd.read_csv('sample_submission.csv')

In [6]:
train_df = sklearn.utils.shuffle(train_df, random_state=4).reset_index(drop=True)

In [7]:
# # converting into Squad format
# def convert_answers(row):
#     return {'answer_start': [row[0]], 'text': [row[1]]}

# train_df['answers'] = train_df[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

In [8]:
train_df = train_df.loc[:2]
# train_df

In [9]:
train_df

,id,context,question,answer_text,answer_start,language
0,5179ed725,ரேடியம் (Radium) என்பது Ra என்ற மூலக்கூற்று வா...,ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?,1898,975,tamil
1,b028b54cf,हेलेन एडम्स केलर (27 जून 1880 - 1 जून 1968) एक...,हेलेन केलर की मृत्यु किस वर्ष में हुई थी?,1968,38,hindi
2,86eff66f2,फ़ूड एण्ड ड्रग एडमिनिस्ट्रेशन (FDA या USFDA) स...,खाद्य एवं औषधि प्रशासन का मुख्यालय कहाँ पर है?,"सिल्वर स्प्रिंग, मैरीलैंड",1092,hindi


In [10]:
# for idx in range(len(train_df)):
#     if train_df.loc[idx,'language'] == "hindi" and len(tokenizer(train_df.loc[idx, 'context'], train_df.loc[idx, 'question'])['input_ids']) > hyperparams.max_len:
#         break

In [11]:
out = tokenizer(
    list(train_df['question']), list(train_df['context']),
    max_length=hyperparams.max_len, 
    truncation='only_second',
    stride=hyperparams.overlap_len,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding=True,
)

In [12]:
print(out.keys())
print(len(out['input_ids']), len(out['offset_mapping']))

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])
62 62


In [13]:
print(len(out['offset_mapping']))
# for i in range(len(out['offset_mapping'])):
#     print(len(out.sequence_ids(i)))

62


In [14]:
print(tokenizer.cls_token, tokenizer.bos_token, tokenizer.padding_side)

<s> <s> right


In [15]:
print(len(out['input_ids']))

62


In [16]:
# print(tokenizer.decode(out['input_ids'][0]))
# print(tokenizer.decode(out['input_ids'][1]))
# print(tokenizer.decode(out['input_ids'][2]))

In [17]:
map_x2context_idx = out['overflow_to_sample_mapping']
print(map_x2context_idx, type(map_x2context_idx))

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] <class 'list'>


In [18]:
map_offset = out['offset_mapping']

In [19]:
idx = random.randint(0, len(out['input_ids']))
idx

36

In [20]:
x_idx = out['input_ids'][idx]
x_context_idx = map_x2context_idx[idx]
print(idx, x_context_idx, len(x_idx))

36 2 384


In [21]:
x_x1_x2 = out.sequence_ids(idx) # sequence ids

In [22]:
# tokenizer.decode(x_idx)

In [23]:
answer_start = train_df.loc[x_context_idx, 'answer_start']
answer = train_df.loc[x_context_idx, 'answer_text']
print(answer)
print(answer_start)

सिल्वर स्प्रिंग, मैरीलैंड
1092


In [24]:
cls_idx = x_idx.index(tokenizer.cls_token_id)
cls_idx

0

In [25]:
y_start_char = train_df.loc[x_context_idx, 'answer_start']
y_end_char = train_df.loc[x_context_idx, 'answer_start'] + len(train_df.loc[x_context_idx, 'answer_text']) # note it is one char ahead

### Data pipeline

In [ ]:
def process_chaii(data_df, tokenizer, train, test):
    # strip trailing and leading whitespaces in context, question, and (answer_text)?
    data_df.loc[:, 'context'] = data_df.loc[:, 'context'].apply(lambda sen : str(sen).strip())
    data_df.loc[:, 'question'] = data_df.loc[:, 'question'].apply(lambda sen : str(sen).strip())
    if test == False:
        data_df.loc[:, 'answer_text'] = data_df.loc[:, 'answer_text'].apply(lambda sen : str(sen).strip())
    
    

In [ ]:
class chaii_ka_data(Dataset):
    def __init__(self, data_df, tokenizer, train=True, test=False):
        super(chaii_ka_data, self).__init__()
        '''
            train = True means train, train = False means val set; test = True means test set (without labels)
            data_df is the pandas dataframe containing context, question, ...        
        '''
        self.train = train
        self.test = test
        
        # tokenize data samples context;question, and create new samples if overflow
        self.data = process_chaii(data_df, tokenizer, train, test)
    
    def __getitem__(self, index):
        pass
    
    def __len__(self):
        pass

### model

### training

### evaluation

### references
1. https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
2. https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.__call__